# Exercises 3.

*Antti Härkönen*

## 1.

$$ q_j*(z_j) \propto exp(E_{-j}[log p(z_j, \bf z_{-j}, \bf x)]) $$

$$ = exp[E_{-j}(log [p(z_j \mid \bf z_{-j}, \bf x) \prod_{i \neq j}p(z_i \mid \bf x) \prod_j p(\bf x_j)])] $$

$$ = exp[E_{-j}(log [p(z_j \mid z_{-j}, x)] + \sum_{i \neq j} E_{i}log [p(z_i \mid x)] + \sum_j E_{j}log [p(\bf x_j)])] $$

$$ = exp(E_{-j}[log p(z_j \mid z_{-j}, x)]) \prod_{i \neq j} exp(E_{i}log [p(z_i \mid x)]) \prod_j exp [E_{j}log p(\bf x_j)] $$

$$ = exp(E_{-j}[log p(z_j \mid z_{-j}, x)]) \prod_{i \neq j} exp(const.) \prod_j exp (const.) $$

$$ \propto exp(E_{-j}[log p(z_j \mid \bf z_{-j}, \bf x)])$$

$$ \square $$

## 2.

$$ q(\mu, \bf \Lambda, \pi) = q(\mu \mid \Lambda, \pi) q(\Lambda \mid \pi) q(\pi) $$

$$ = q(\pi) \prod^K_{k=1} q(\mu_k \mid \Lambda_k, \pi) \prod^K_{k=1} q(\Lambda_k \mid \pi) $$

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k, \pi)}{q(\Lambda_k, \pi)} \times \frac{q(\Lambda_k, \pi)}{q(\pi)} $$

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k, \pi)}{q(\pi)} $$

There are no paths from $\pi$ to $\mu$ or $\Lambda$, so they are independent and
$q(\mu_k, \Lambda_k, \pi) = q(\mu_k, \Lambda_k) q(\pi)$.

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k) q(\pi)}{q(\pi)} $$

$$ = q(\pi) \prod^K_{k=1}q(\mu_k, \Lambda_k) $$

$$ \square $$

## 3.

In [28]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.special import psi

edges = pd.read_csv('email-Eu-core.txt', sep=' ')
labels = pd.read_csv('email-Eu-core-department-labels.txt', sep=' ')
y = nx.to_numpy_matrix(nx.from_pandas_edgelist(edges, '0', '1'))

class MMSBModel:
    def __init__(self, y, k, alpha):
        self.N = self.M = y.shape[0]
        self.K = k
        self.alpha = np.repeat(alpha, self.K)
        self.Y = y
        self.phi = np.random.dirichlet(self.alpha, size=(self.N, self.M))
        self.gamma = np.random.dirichlet(np.ones(self.K), size=self.N)
        self.B = np.random.random(size=(self.K, self.K))
        self.rho = (1. - np.sum(self.Y) / (self.N * self.M))

    def vi(self, convergence_threshold=0.01, max_iter=1000):
        Y = np.array(self.Y, copy=True)[..., np.newaxis, np.newaxis]
        for it in range(max_iter):
            phi = np.broadcast_to(self.phi[..., np.newaxis], (self.N, self.M, self.K, self.K)).copy()
            phi[np.arange(self.N), np.arange(self.M)] = np.zeros((self.K, self.K))
            possible_connections = np.transpose(phi, axes=(1, 0, 3, 2)) * phi
            B_new = (possible_connections * np.broadcast_to(
                        self.Y[..., np.newaxis, np.newaxis],
                        possible_connections.shape)
                    ).sum(axis=(0, 1)) / possible_connections.sum(axis=(0, 1))

            for _ in range(max_iter):
                phi = self.phi.copy()
                mean_log_pi = psi(self.gamma) - psi(self.gamma.sum(axis=1, keepdims=True))
                mean_log_pi = np.broadcast_to(
                    np.expand_dims(mean_log_pi, axis=1),
                    (self.N, self.M, self.K),
                ).copy()
                phi = mean_log_pi * np.prod(((B_new**Y) * ((1.-B_new)**(1. - Y)))**self.phi)

                phi_new = phi / phi.sum(axis=2, keepdims=True)
                print(phi_new.shape)

                # estimate gamma

                self.phi = phi_new.copy()
                change_gamma = (gamma_new - self.gamma).abs().sum()
                self.gamma = gamma_new
                if change_gamma <= convergence_threshold:
                    break
            change_B = np.abs((B_new - self.B)).sum()
            self.B = B_new
            if change_B <= convergence_threshold:
                print(f"VI finished with {it} iterations.")
                break
        print(f"VI finished with {max_iter} iterations.")


model = MMSBModel(y[:100,:100], 5, 0.01)
model.vi()

ValueError: operands could not be broadcast together with shapes (100,100,5,5) (100,100,5) 

## 4.